In [2]:
# 查看GPU使用情况
! pip install pynvml
import pynvml
import torch
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0) # 0表示显卡标号
device_count=torch.cuda.device_count()
print(device_count)
# 在每一个要查看的地方都要重新定义一个meminfo 
meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
print(meminfo.total/1024**2) #总的显存大小
print(meminfo.used/1024**2)  #已用显存大小
print(meminfo.free/1024**2)  #剩余显存大小
# 单位是MB，如果想看G就再除以一个1024


Defaulting to user installation because normal site-packages is not writeable
  Using cached pynvml-11.5.0-py3-none-any.whl (53 kB)
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
8
24576.0
316.4375
23.69097900390625


In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
! pip install sentencepiece

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = AutoModel.from_pretrained("/home/grammar/Paper_Corrector/backend_interact/mymodel", trust_remote_code=True).to(device).float()
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained("/home/grammar/Paper_Corrector/backend_interact/mymodel", trust_remote_code=True)

model = model.eval()

response, history = model.chat(tokenizer, "你有什么要对我说的吗", history=[])
print(response)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

如果你有任何问题或需要帮助,请随时告诉我。我是一个计算机程序,没有任何情感或意识,但我可以提供最准确的信息和帮助解决问题。


In [4]:
response, history = model.chat(tokenizer, "请判断如下句子是否是病句，如果是，请判断错误的类别并且返回修改后的结果。句子： 做科学上的事实判断，不同的人从不同的视角、不同的方式为出发点，终究可以获得一致的结论，达成某种程度的一致的意见。", history=[])
print(response)

RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

**下面这些代码在下面的markdown特殊声明之前都不要跑！** 这里是用CLG生成数据作测试集测试模型性能的部分，使用的评价方式是字符串完全匹配。

In [ ]:
import csv
import json

jsonfile=open('resp.json','w',encoding='gb2312')
count=1
notwritten=[]
with open('dataset.csv', 'r',encoding='gb2312') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        text=row[1]
        # print(text)
        with open('prompt.txt','r',encoding='utf-8')as f:
            prompt=f.read()
            # print(prompt)
            prompt=prompt+f"\n#\n{text}\n#"
            # print(prompt)
            response, history = model.chat(tokenizer, prompt, history=[])
            # print(response)
            try:
                respjson=json.loads(response)
                
            except:
                print('not written!')
                notwritten.append(count)
                pass
            json.dump(respjson,jsonfile)
            jsonfile.write(',')
            print(count)
            count+=1
    jsonfile.close()
        

In [23]:
jsonfile.close()

In [4]:
import json
with open('resp.json','r',encoding='utf-8')as f:
    data=json.load(f)
with open('resp.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [23]:
with open('resp.csv','w',encoding='utf-8') as f:
    writer=csv.writer(f)
    writer.writerow(data[0].keys())
    for item in data:
        writer.writerow(item.values())

modify&test:

In [12]:
import pandas as pd
import csv
content=[]
with open('resp.csv', 'r') as f:
    reader = csv.reader(f)
    # print(type(reader))
    header = next(reader) # 读取表头
    num_fields = len(header) # 字段数目
    content.append(header)
    for row in reader:
        # print(type(row))
        if len(row) != num_fields:
            print(f"Error: Expected {num_fields} fields in line {reader.line_num}, saw {len(row)}")
            del row[2]
            
            print('line ',reader.line_num,'has been modified!')
        row[0]=row[0].strip()
        row[1]=row[1].strip()
        row[2]=row[2].strip()
        content.append(row)
with open('resp.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(content)


transform encoding

In [2]:
import codecs
import csv
with open('dataset.csv', 'r', encoding='gb2312') as f:
    reader = csv.reader(f)
    rows=[row for row in reader]

with codecs.open('dataset.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(rows)


UnicodeDecodeError: 'gb2312' codec can't decode byte 0xe5 in position 6: illegal multibyte sequence

In [6]:
rows=[]
with open('dataset.csv', 'r') as f:
    reader=csv.reader(f)
    for row in reader:
        row[1]=row[1].strip()
        row[2]=row[2].strip()
        rows.append(row)
with open('dataset.csv', 'w') as f:
    writer= csv.writer(f)
    writer.writerows(rows)

In [13]:
import pandas as pd
df1=pd.read_csv('dataset.csv',low_memory=False)
print(df1)
# for i in df1:
#     print(i)
# print(df1.keys)
df2=pd.read_csv('resp.csv')
print(df2)
# for i in df2:
    # print(i)
df=pd.merge(df2,df1,on="病句",how='left')
df.to_csv('test.csv', encoding="utf_8_sig")

                                                     正确句子  \
0       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
1       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
2       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
3       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
4       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...   
...                                                   ...   
222609         "[20]（p.39）但随着研究的不断深入，学者们得出的结论也逐渐趋向比较客观和公允   
222610         "[20]（p.39）但随着研究的不断深入，学者们得出的结论也逐渐趋向比较客观和公允   
222611                      "、"白身"，把缺乏锻炼、阅历不深的文人称作"白面书生"等   
222612                      "、"白身"，把缺乏锻炼、阅历不深的文人称作"白面书生"等   
222613                      "、"白身"，把缺乏锻炼、阅历不深的文人称作"白面书生"等   

                                                       病句  语病类型  Unnamed: 3  \
0       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...  成分残缺         NaN   
1       做哲学上的价值判断，每个人都是根据自己的标准来进行，判断的标准是因人而异的，甚至同一个人对同...  成分残缺         NaN   
2       做哲学上的价值判断，每个人都是根据自己的标准

莫名其妙就能跑了。。。

In [14]:
with open('test.csv','r')as f:
    judge_wrong=0
    modify_wrong=0
    tot_num=0
    reader = csv.reader(f)
    header=next(reader)
    for row in reader:
        if row[2]!=row[5]:
            judge_wrong+=1
        if row[3]!=row[4]:
            modify_wrong+=1
        tot_num+=1
jug_acc=1-judge_wrong/tot_num
mod_acc=1-modify_wrong/tot_num
print("judging accuracy:",jug_acc)
print("modifying cohesion:",mod_acc)
        

judging accuracy: 0.044487427466150864
modifying cohesion: 0.0


由于代码的特殊性，上面所作测试代码全部保留，之后再次运行的时候只需要执行前面两个单元格就可以了，之后直接执行这下面的代码。接下来要做的是使用NaCGEC的验证集直接对模型做测试，测试的标准是M2-SCORER。

In [1]:
import pandas as pd
data=[]
with open('CGEDdataset/CGED_test.txt','r',encoding='utf-8')as f:
    for line in f:
        data.append(line.strip())
# dataset=pd.DataFrame(data,columns=['Id','original','corrected1','corrected2','corrected3','corrected4'])
# dataset.head()
data[:20]

['16岁时我高中退学了，当时我不知我要做什么才好，天天打工赚钱，就去玩儿花钱。',
 '1940年8月11日，我出生在印度尼西亚西爪哇的万隆市。',
 '1945年，日本投降时，我和妈妈、哥哥还留在马来西亚的深山。',
 '1950年10月9日，我来到了这个五花八门的世界。',
 '1952年3月3日，我出生于印度尼西亚的一个小镇。',
 '1973年亲属们一知道我母亲生了一个女孩时，邻居们份份的来看父亲，表示自己对他的同情。',
 '1988年，我复学以后组织了一个学习小组。',
 '1989、1990、1991年，他获得最优秀推销员赏。',
 '1990年，回本国来继续研究世界有名的名胜古迹、世界各国的历史，而且对于人类发展史和文明、生活风俗习惯有很厚的知识，为了打下我的经验，1992年到1993年1年间我去世界旅行。',
 '1992年至今：任印尼邪加达信心旅游公司业务部经理。',
 '1993年，我考入天津大学，修读美术系，并在1995年毕业，获得美术学位。',
 '1994年4月17号是我父亲离开我跟我家里人的暗天。',
 '1994年初发生在中国农村的这件事震惊。妻子得了一种不治之症，十分痛苦，要求丈夫帮助她自杀，使用安乐死来结束生命。',
 '1994年初，在中国农村，发生了这样一件事：妻子得了一种不治之症，十分痛苦，要求她的丈夫帮助她自杀（即安乐而死）。',
 '1999年1月，我从大学完学了。',
 '1999年9月，我进了北京中医药大学。',
 '2004年春天的有一天，我和女朋友一起逛街的时候，我听到了有一个好听的歌，就是《2002年的第一场雪》，特别好听，我越听越好。',
 '21世纪的亚洲会遭到香烟的“攻击”。',
 '如果找不到垃圾桶应保管好，等找到垃圾桶后再丢。',
 '以下是我本人的简历。姓名：彭莉莉。姓别：女。年龄：二十三岁。未婚。中学：尊孔国民型中学。工人经验：二年。成绩：曾举办过服装展，参加过毕业服装设计展，以“优良”成绩毕业。']

In [17]:
originals=dataset.original.tolist()
originals[:5]

['2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。',
 '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。',
 '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。',
 '热播剧《山海情》的故事发生在宁夏西海固地区。为了保护当地生态环境和群众生活，20世纪八十年代，当地政府组织6万余名西海固人陆续从大山深处搬迁扎根到宁夏平原的闽宁镇，走上了脱贫致富之路。',
 '老同志不但关心我，就是和他一起入伍的新同志也经常帮助我，这使我非常感动。']

In [6]:

# print(corrections)
# print(prompt)
corrections=[]
count=1
for origin in data:
    prompt="""你的任务是基于接下来提供的"原始病句"给出修改后的正确句子。"原始病句"将会以三个反引号包裹起来给出。\
病句需要修改的地方只会出现以下语病中的一种或几种：Structural Confusion,Improper Logicality,Missing Component,Redundant Component,Improper Collocation,Improper Word Order.\n
你的回复遵循下面这种格式:\n
{
    'origin':'...',
    'corrected':'...'
}
其中'origin'后面照抄接下来提供的病句,'corrected'后面放你的改句，如果你认为没有语病，则复制"原始病句".
\n原始病句："""
    prompt=prompt+"```"+origin+"```"
    # print(prompt)
    response, history = model.chat(tokenizer, prompt, history=[],temperature=0.01)
    print("generated sentence ",count)
    # print(response)
    count+=1
    corrections.append(response)


generated sentence  1
generated sentence  2
generated sentence  3
generated sentence  4
generated sentence  5
generated sentence  6
generated sentence  7
generated sentence  8
generated sentence  9
generated sentence  10
generated sentence  11
generated sentence  12
generated sentence  13
generated sentence  14
generated sentence  15
generated sentence  16
generated sentence  17
generated sentence  18
generated sentence  19
generated sentence  20
generated sentence  21
generated sentence  22
generated sentence  23
generated sentence  24
generated sentence  25
generated sentence  26
generated sentence  27
generated sentence  28
generated sentence  29
generated sentence  30
generated sentence  31
generated sentence  32
generated sentence  33
generated sentence  34
generated sentence  35
generated sentence  36
generated sentence  37
generated sentence  38
generated sentence  39
generated sentence  40
generated sentence  41
generated sentence  42
generated sentence  43
generated sentence  

KeyboardInterrupt: 

In [10]:
corrections 
with open('CGED_sys.txt','w',encoding='utf-8')as f:
    for co in corrections:
        f.write(co)
        f.write('\n')

In [1]:
import re
corrections[0]="{\n    'origin':'...',\n    'corrected':'2017年《财富》全球论坛落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又充满活力的广州。'\n}"
grammar=re.compile(r"'corrected':'(?P<content>.*?)'|改进后的句子:```(?P<content>.*?)`",re.S)
for i in range(0,len(corrections)):
    correction=re.search(grammar,corrections[i])
    print(correction.group('content'))
    # break

NameError: name 'corrections' is not defined